In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now().date()} / Aki Taanila')

Päivitetty 2025-11-01 / Aki Taanila


# Pika-analyysi Exceliin

Frekvenssitaulukot, ristiintaulukoinnit, keskeiset tunnusluvut ja korrelaatiot Exceliin.

Jos **xlwings**-kirjasto ei ole valmiiksi asennettu, niin voit asentaa sen komentorivillä (pääte/terminaali) komennolla `conda install xlwings`.

Pika-analyysiä voit käyttää seuraavasti:

* Avaa Exceliin analysoitava data ja valitse yksi solu datan alueelta
* Siirry tähän muistioon ja suorita koodi
* Koodin suorittamisen jälkeen tulokset löytyvät avoinna olevasta uudesta Excel-tiedostosta.

In [2]:
import pandas as pd
import xlwings as xw

# Jos muuttujalla on ainutkertaisia arvoja yli RAJAn, niin muuttuja tulkitaan määrälliseksi
# RAJA-arvoa voit tarvittaessa muuttaa
RAJA = 10

# Datan voi napata suoraan auki olevasta Excel-tiedostosta:
df = xw.load(index=False)
# Jos et halua datan ensimmäistä saraketta analyyseihin, niin käytä mieluummin komentoa:
#df = xw.load()

In [3]:
# Alustavat toimet

# luodaan Listat määrällisistä ja kategorisista muuttujista
kvantit = []
kategoriset = []
for muuttuja in df:
    if (len(df[muuttuja].unique())>RAJA) & (df[muuttuja].dtype in ['int64', 'float64']):
        kvantit.append(muuttuja)
    elif len(df[muuttuja].unique())<=RAJA:
        kategoriset.append(muuttuja)


# Excelin valmistelu

# Piilotettu Excel-instanssi
app = xw.App(visible=False)

# Taulukkovälilehdet Excelissä
wb = app.books[0]
ws1= wb.sheets.active
ws1.name = 'frekvenssitaulukot'
ws2 = wb.sheets.add('ristiintaulukoinnit', after=wb.sheets.count)
ws3 = wb.sheets.add('tunnusluvut', after=wb.sheets.count)
ws4 = wb.sheets.add('korrelaatiot', after=wb.sheets.count)

In [4]:
# Frekvenssitaulukot

if kategoriset:
    # Pidän kirjaa Excelin rivinumerosta rivi-muuttujan avulla
    rivi = 1
    sarake = 1
    # Käyn for-silmukalla läpi kaikki kategoriset muuttujat
    for muuttuja in kategoriset:
        # Lasken frekvenssit df1-nimiseen dataframeen
        df1 = pd.crosstab(df[muuttuja], 'f')
        # Lasken df1:een prosentit
        df1['%'] = df1/df1.sum()
        # Lisään df1:een Yhteensä-rivin
        df1.loc['Kaikki'] = df1.sum()
        
        # Kirjoitan frekvenssitaulukon Excel-tiedoston Frekvenssit-taulukkovälilehdelle
        ws1.range((rivi, sarake)).value = df1
        
        # Muotoilut
        ws1.range((rivi+1, sarake+2),(rivi+len(df1), sarake+2)).number_format = '0,0 %'
        ws1.range((rivi, sarake+1), (rivi, sarake+2)).api.HorizontalAlignment = -4152
        ws1.range((rivi, sarake), (rivi, sarake+2)).api.Borders(9).Weight = 2
        ws1.range((rivi+len(df1)-1, sarake), (rivi+len(df1)-1, sarake+2)).api.Borders(9).Weight = 2
        
        # Kasvatan rivinumeroa; shape[0] antaa df1:n rivimäärän
        rivi = rivi+df1.shape[0]+2

In [5]:
# Ristiintaulukoinnit
    
if len(kategoriset) > 1:
    rivi = 1
    sarake = 1
    for muuttuja1 in kategoriset:
        for muuttuja2 in kategoriset:
            if muuttuja1 != muuttuja2:
                df1 = pd.crosstab(df[muuttuja1], df[muuttuja2])
                df2 = pd.crosstab(df[muuttuja1], df[muuttuja2], normalize='columns')
                df2.index.name = muuttuja1+'/'+muuttuja2 
                df2.loc['n'] = df1.sum()
                
                # Kirjoitan ristiintaulukoinnin Exceliin
                ws2.range((rivi, sarake)).value = df2
                
                # Muotoilut
                ws2.range((rivi+1, sarake+1),(rivi+len(df1), sarake+df1.shape[1])).number_format = '0,0 %'
                ws2.range((rivi, sarake+1), (rivi, sarake+df1.shape[1])).api.HorizontalAlignment = -4152
                ws2.range((rivi, sarake), (rivi, sarake+df1.shape[1])).api.Borders(9).Weight = 2
                ws2.range((rivi+len(df1), sarake), (rivi+len(df1), sarake+df1.shape[1])).api.Borders(9).Weight = 2
                                    
                # Kasvatan rivinumeroa
                rivi = rivi+df2.shape[0]+2

In [6]:
# Tunnusluvut

# Tunnusluvut suomeksi
tunnusluvut = ['Lukumäärä', 'Keskiarvo', 'Keskihajonta', 'Pienin', 
              'Alaneljännes', 'Mediaani', 'Yläneljännes', 'Suurin']
            
if kvantit:
    rivi = 1
    sarake = 1
    df1 = df[kvantit].describe()
    
    # Tunnusluvut suomeksi
    df1.index = tunnusluvut
    
    # Kirjoitan taulukon Exceliin
    ws3.range(rivi, sarake).value = df1

    # Sarakeleveyden säätö
    ws3.range('A:A').autofit()

    # Muotoilut
    ws3.range((rivi+2, sarake+1),(rivi+len(df1), sarake+df1.shape[1])).number_format = '0,0'
    ws3.range((rivi, sarake+1), (rivi, sarake+df1.shape[1])).api.HorizontalAlignment = -4152
    ws3.range((rivi, sarake), (rivi, sarake+df1.shape[1])).api.Borders(9).Weight = 2
    ws3.range((rivi+len(df1), sarake), (rivi+len(df1), sarake+df1.shape[1])).api.Borders(9).Weight = 2

    
# Tunnusluvut kategoristen määräämissä ryhmissä

if kategoriset:
    rivi = df1.shape[0]+3
    for muuttuja1 in kategoriset:
        for muuttuja2 in kvantit:
            if muuttuja1 != muuttuja2:
                df1 = df.groupby(muuttuja1)[muuttuja2].describe()
                
                # Tunnusluvut suomeksi
                df1.columns = tunnusluvut
                
                df1.index.name = muuttuja1+'/'+muuttuja2
                    
                # Kirjoitan taulukon Exceliin
                ws3.range(rivi, sarake).value = df1
                
                # Muotoilut
                ws3.range((rivi+1, sarake+2),(rivi+len(df1), sarake+df1.shape[1])).number_format = '0,0'
                ws3.range((rivi, sarake+1), (rivi, sarake+df1.shape[1])).api.HorizontalAlignment = -4152
                ws3.range((rivi, sarake), (rivi, sarake+df1.shape[1])).api.Borders(9).Weight = 2
                ws3.range((rivi+len(df1), sarake), (rivi+len(df1), sarake+df1.shape[1])).api.Borders(9).Weight = 2
                    
                # Kasvatan rivinumeroa
                rivi = rivi+df1.shape[0]+2

# Sarakeleveyksie säätöä
ws3.range('B:I').column_width = 11

In [7]:
# Korrelaatiot                
                
if kvantit:
    rivi = 1
    range = 1
    df1 = df[kvantit].corr()
    
    # Kirjoitan taulukon Exceliin
    ws4.range(rivi, sarake).value = df1
    
    # Muotoilut
    ws4.range((rivi+1, sarake+1),(rivi+len(df1), sarake+df1.shape[1])).number_format = '0,000'
    ws4.range((rivi, sarake+1), (rivi, sarake+df1.shape[1])).api.HorizontalAlignment = -4152
    ws4.range((rivi, sarake), (rivi, sarake+df1.shape[1])).api.Borders(9).Weight = 2
    ws4.range((rivi+len(df1), sarake), (rivi+len(df1), sarake+df1.shape[1])).api.Borders(9).Weight = 2

In [8]:
# Excel-instanssi näkyville
ws1.activate()
app.visible = True

Lisätietoa Pythonin käytöstä data-analytiikassa https://tilastoapu.wordpress.com/python/